# Getting data

In [1]:
import cv2
import json
from json import JSONEncoder
import pandas as pd
import numpy as np
import pycocotools.mask as mask_util

from tqdm import tqdm

In [2]:
with open('/home/ubuntu/dormakaba/coco/annotations/instances_val2014.json') as json_file:
    data = json.load(json_file)

In [3]:
with open('/home/ubuntu/dormakaba/coco/annotations/densepose_coco_2014_valminusminival.json') as json_file:
    person_data = json.load(json_file)

In [4]:
data_df = pd.json_normalize(data)

In [5]:
person_data_df = pd.json_normalize(person_data)

In [6]:
data_df.keys()

Index(['images', 'licenses', 'annotations', 'categories', 'info.description',
       'info.url', 'info.version', 'info.year', 'info.contributor',
       'info.date_created'],
      dtype='object')

In [7]:
person_data_df.keys()

Index(['images', 'annotations', 'categories'], dtype='object')

In [8]:
# esta es la lista de imágenes anotadas con personas
# está ok
person_data_df['images'][0][0].keys()

dict_keys(['license', 'file_name', 'coco_url', 'height', 'width', 'date_captured', 'flickr_url', 'id'])

In [9]:
print(person_data_df['categories'][0])

[{'supercategory': 'person', 'id': 1, 'keypoints': ['nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear', 'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 'left_hip', 'right_hip', 'left_knee', 'right_knee', 'left_ankle', 'right_ankle'], 'name': 'person', 'skeleton': [[16, 14], [14, 12], [17, 15], [15, 13], [12, 13], [6, 12], [7, 13], [6, 7], [6, 8], [7, 9], [8, 10], [9, 11], [2, 3], [1, 2], [1, 3], [2, 4], [3, 5], [4, 6], [5, 7]]}]


# Generating classes

In [10]:
COCO_CLASSES = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
                'train', 'truck', 'boat', 'traffic light', 'fire hydrant',
                'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog',
                'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe',
                'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
                'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat',
                'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
                'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
                'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot',
                'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
                'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop',
                'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven',
                'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase',
                'scissors', 'teddy bear', 'hair drier', 'toothbrush']
CUSTOM_CLASSES = ['person', 'bicycle', 'cat', 'dog', 'backpack', 'umbrella', 'handbag', 'suitcase', 
                'sports ball', 'baseball bat', 'skateboard', 'tennis racket', 'bottle', 'wine glass', 
                'sandwich', 'chair', 'laptop', 'cell phone', 'book', 'clock', 'scissors', 'teddy bear',
                'torso', 'hand1', 'hand2', 'foot1', 'foot2', 'upleg1', 'upleg2', 'lowleg1', 'lowleg2',
                'uparm1', 'uparm2', 'lowarm1','lowarm2', 'head']
FROM_COCO_CLASSES = ['person', 'bicycle', 'cat', 'dog', 'backpack', 'umbrella', 'handbag', 'suitcase', 
                'sports ball', 'baseball bat', 'skateboard', 'tennis racket', 'bottle', 'wine glass', 
                'sandwich', 'chair', 'laptop', 'cell phone', 'book', 'clock', 'scissors', 'teddy bear']
NEW_CLASSES = ['torso', 'hand1', 'hand2', 'foot1', 'foot2', 'upleg1', 'upleg2', 'lowleg1', 'lowleg2',
                'uparm1', 'uparm2', 'lowarm1','lowarm2', 'head']
COCO_LABEL_MAP = {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8,
                  9: 9, 10: 10, 11: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16,
                  18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24,
                  27: 25, 28: 26, 31: 27, 32: 28, 33: 29, 34: 30, 35: 31, 36: 32,
                  37: 33, 38: 34, 39: 35, 40: 36, 41: 37, 42: 38, 43: 39, 44: 40,
                  46: 41, 47: 42, 48: 43, 49: 44, 50: 45, 51: 46, 52: 47, 53: 48,
                  54: 49, 55: 50, 56: 51, 57: 52, 58: 53, 59: 54, 60: 55, 61: 56,
                  62: 57, 63: 58, 64: 59, 65: 60, 67: 61, 70: 62, 72: 63, 73: 64,
                  74: 65, 75: 66, 76: 67, 77: 68, 78: 69, 79: 70, 80: 71, 81: 72,
                  82: 73, 84: 74, 85: 75, 86: 76, 87: 77, 88: 78, 89: 79, 90: 80}

In [11]:
REVERSE_COCO_LABEL_MAP = {}
for key, value in COCO_LABEL_MAP.items():
    REVERSE_COCO_LABEL_MAP[value] = key

In [12]:
print(data_df['categories'][0][0])

custom_categories = []
for cat_data in data_df['categories'][0]:
    if cat_data['name'] in CUSTOM_CLASSES:
        custom_categories.append(cat_data)

for nc in NEW_CLASSES:
    c = {'supercategory': 'person', 'id': 1, 'name': nc}
    custom_categories.append(c)

cc_id = 1
cc_list = [0]*37
for cc in custom_categories:
    cc['id'] = cc_id
    cc_list[cc_id] = cc['name']
    cc_id += 1

# this is the list of the new categories
print(custom_categories)
# the list of the names of the categories
print(cc_list)

{'supercategory': 'person', 'id': 1, 'name': 'person'}
[{'supercategory': 'person', 'id': 1, 'name': 'person'}, {'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'}, {'supercategory': 'animal', 'id': 3, 'name': 'cat'}, {'supercategory': 'animal', 'id': 4, 'name': 'dog'}, {'supercategory': 'accessory', 'id': 5, 'name': 'backpack'}, {'supercategory': 'accessory', 'id': 6, 'name': 'umbrella'}, {'supercategory': 'accessory', 'id': 7, 'name': 'handbag'}, {'supercategory': 'accessory', 'id': 8, 'name': 'suitcase'}, {'supercategory': 'sports', 'id': 9, 'name': 'sports ball'}, {'supercategory': 'sports', 'id': 10, 'name': 'baseball bat'}, {'supercategory': 'sports', 'id': 11, 'name': 'skateboard'}, {'supercategory': 'sports', 'id': 12, 'name': 'tennis racket'}, {'supercategory': 'kitchen', 'id': 13, 'name': 'bottle'}, {'supercategory': 'kitchen', 'id': 14, 'name': 'wine glass'}, {'supercategory': 'food', 'id': 15, 'name': 'sandwich'}, {'supercategory': 'furniture', 'id': 16, 'name': 'chair

In [13]:
coco_category_id_list = [0]*len(cc_list)
for cat in data_df['categories'][0]:
    if cat['name'] in cc_list:
        pos = cc_list.index(cat['name'])
        coco_category_id_list[pos] = cat['id']

# this is the list of the COCO ids of the new categories (when exists)
print(coco_category_id_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [14]:
# this variable has the coco category id in the position of the custom category
custom_category_coco_category = [0]*len(cc_list)
for i, cat_name in enumerate(COCO_CLASSES):
    if cat_name in cc_list:
        cc_pos = cc_list.index(cat_name)
        coco_cat_id = REVERSE_COCO_LABEL_MAP[i+1]
        custom_category_coco_category[cc_pos] = coco_cat_id
        # print(i, cat_name, cc_pos, coco_cat_id)

print(custom_category_coco_category)

[0, 1, 2, 17, 18, 27, 28, 31, 33, 37, 39, 41, 43, 44, 46, 54, 62, 73, 77, 84, 85, 87, 88, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


# looking at images

In [15]:
new_person_images = person_data_df['images'][0]
print( len(new_person_images), new_person_images[0].keys())

5984 dict_keys(['license', 'file_name', 'coco_url', 'height', 'width', 'date_captured', 'flickr_url', 'id'])


In [16]:
images_person_data_df = pd.DataFrame(person_data_df['images'][0])

In [17]:
# a = images_person_data_df[images_person_data_df['id']==262145]
# print(a['height'][0], a['width'][0])

In [18]:
person_data_df['images'][0][0]

{'license': 1,
 'file_name': 'COCO_val2014_000000109229.jpg',
 'coco_url': 'http://mscoco.org/images/109229',
 'height': 640,
 'width': 514,
 'date_captured': '2013-11-18 17:27:34',
 'flickr_url': 'http://farm3.staticflickr.com/2342/1794334693_c3e27b61dd_z.jpg',
 'id': 109229}

In [19]:
# list of image ids from human parts data
new_person_images_list = [a['id'] for a in person_data_df['images'][0]]
len(new_person_images_list)

5984

In [20]:
all_images_list = [a['id'] for a in data_df['images'][0]]
len(all_images_list)

40504

# looking at annotations

In [21]:
# person annotations
# extraer segmentation: anotaciòn individual de cada persona detectada
# extrater dp_masks: e insertar una anotación por cada uno de los 14 elementos de la persona que tienen anotaciones
# si se puede, transformar el mapa de bits en 
print(person_data_df['annotations'][0][0].keys())
print(person_data_df['annotations'][0][0]['dp_masks'])

dict_keys(['segmentation', 'num_keypoints', 'dp_masks', 'area', 'dp_I', 'iscrowd', 'keypoints', 'dp_x', 'dp_U', 'image_id', 'dp_V', 'bbox', 'category_id', 'dp_y', 'id'])
[{'counts': '[bh03m70O5I8I7B=D=K=C;E0000001N101O1N101N101O000010O010O1O01O010O010O001O010O1O010N1000O100O001O100O1O01O0100O010OO12O000000001O000000001O000O12NJ7D;ZNPJR1NjI8l6Hf0O2O0O2O0000XVc0', 'size': [256, 256]}, {'counts': 'mQ11o72N000O1000001N0100000O100O1O0100000O100O10O10010O01O1O001OO1O2N1N3M2O10Wfe1', 'size': [256, 256]}, {'counts': 'Pkj1173]7:N0O3N1N2N010O02O0O2N101M2M5LQU1', 'size': [256, 256]}, {'counts': 'ZgQ16g77L3M1O001O001OO10000000O14L5K4L4LYhi0', 'size': [256, 256]}, {'counts': 'j_l01o72N2N1N1000000000000000000O1000000O12N3M3MQPn0', 'size': [256, 256]}, {'counts': 'U\\j0h1S64N21O101N1O2N2O001O001O00001O001O000010O100O100O101NN3ROn0I6J7Ikcm0', 'size': [256, 256]}, {'counts': 'P]Q1a0Y7d0\\O5K6M210O10O01O002O0O2N101N101N103L3MN3N2N2M3N200N2I7F:I7K5J6K4IXTf0', 'size': [256, 256]}, {'counts': 'm]j09f7b0^O<

In [22]:
print(person_data_df['annotations'][0][0])

{'segmentation': [[217.17, 355, 218.61, 349.24, 212.85, 324.79, 214.29, 313.29, 225.8, 296.03, 244.49, 296.03, 256, 304.66, 257.44, 339.18, 257.44, 350.68, 325.03, 382.32, 352.36, 406.77, 345.17, 418.28, 343.73, 422.59, 332.22, 422.59, 312.09, 409.65, 279.01, 396.7, 273.26, 389.51, 264.63, 445.6, 258.88, 534.77, 251.69, 563.54, 240.18, 580.79, 244.49, 587.98, 224.36, 587.98, 208.54, 587.98, 207.1, 587.98, 202.79, 582.23, 201.35, 562.1, 201.35, 541.96, 201.35, 501.69, 202.79, 390.95, 201.35, 382.32, 178.34, 393.83, 171.15, 393.83, 161.08, 390.95, 145.26, 383.76, 135.19, 375.13, 116.49, 369.38, 109.3, 369.38, 117.93, 362.19, 136.63, 359.31, 142.38, 356.43, 143.82, 356.43, 169.71, 370.82, 199.91, 359.31]], 'num_keypoints': 17, 'dp_masks': [{'counts': '[bh03m70O5I8I7B=D=K=C;E0000001N101O1N101N101O000010O010O1O01O010O010O001O010O1O010N1000O100O001O100O1O01O0100O010OO12O000000001O000000001O000O12NJ7D;ZNPJR1NjI8l6Hf0O2O0O2O0000XVc0', 'size': [256, 256]}, {'counts': 'mQ11o72N000O1000001N010000

# generating annotations

In [23]:
def GetDensePoseMasks(Polys):
    mask_list = [0]*15
    for i in range(1,15):
        MaskGen = np.zeros([256,256])
        if(Polys[i-1]):
            current_mask = mask_util.decode(Polys[i-1])
            MaskGen[current_mask>0] = 1
        mask_list[i-1]=MaskGen
    return mask_list

In [24]:
def GetDensePoseMask(encoded_mask):
    MaskGen = np.zeros([256,256])
    current_mask = mask_util.decode(encoded_mask)
    MaskGen[current_mask>0] = 1
    return MaskGen

In [25]:
def get_poligons(mask):
    c = cv2.findContours(mask.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = [0]*2*len(c[0][0].tolist())
    for i,x in enumerate(c[0][0].tolist()):
        contours[2*i]=int(x[0][0])
        contours[2*i+1]= int(x[0][1])
    return contours

In [26]:
def CreateMaskBBox(mask, bbr, image_shape):
    final_mask = np.zeros([image_shape[1],image_shape[0]])
    x1 = int(bbr[0])
    y1 = int(bbr[1])
    x2 = int(bbr[0]+bbr[2])
    y2 = int(bbr[1]+bbr[3])
    x2 = min( [ x2, image_shape[0] ] )
    y2 = min( [ y2, image_shape[1] ] )
    # print(bbr, (x1,y1,x2,y2), (int(x2-x1),int(y2-y1)), mask.shape)
    resized_mask = cv2.resize( mask, (int(x2-x1),int(y2-y1)) ,interpolation=cv2.INTER_NEAREST)
    final_mask[y1:y2,x1:x2] = resized_mask
    az = np.nonzero(final_mask)
    bby1,bby2,bbx1,bbx2=np.min(az[0]), np.max(az[0]), np.min(az[1]), np.max(az[1])
    contours = get_poligons(final_mask.astype(np.uint8))
    encoded_final_mask = mask_util.encode(np.asfortranarray(final_mask.astype(np.uint8)))
    mask_area = int(mask_util.area(encoded_final_mask))
    # print((bbx1, bby1, bbx2-bbx1, bby2-bby1),encoded_final_mask)
    encoded_final_mask['counts'] = encoded_final_mask['counts'].decode("utf-8")
    return final_mask, (bbx1, bby1, bbx2-bbx1, bby2-bby1), mask_area, encoded_final_mask, contours

In [27]:
new_person_ann = []
for annotation in tqdm(person_data_df['annotations'][0]):
    # print(annotation['image_id'], annotation['bbox'])
    if annotation['iscrowd']==0:
        ann_segment = annotation['segmentation']
        ann_bbox = [int(annotation['bbox'][0]),int(annotation['bbox'][1]),int(annotation['bbox'][2]),int(annotation['bbox'][3])]
        ann_area = annotation['area']
        ann_image_id = annotation['image_id']
        ann_id = annotation['id']
        ann_cat_id = 1 # only person cat so must be 1
        new_data = {'segmentation': ann_segment,
                    'area': ann_area,
                    'image_id': ann_image_id,
                    'id': ann_id,
                    'category_id': ann_cat_id,
                    'bbox': ann_bbox,
                    'iscrowd': 0}
        new_person_ann.append(new_data)
        a = images_person_data_df[images_person_data_df['id']==ann_image_id]
        image_size = [a['width'].values[0],a['height'].values[0]]

        if 'dp_masks' in annotation:
            for i, ann_mask in enumerate(annotation['dp_masks']):
                if len(ann_mask) > 1:
                    ann_mask_decoded = GetDensePoseMask(ann_mask)
                    mask, bbox, ann_area, encoded_mask, contours = CreateMaskBBox(ann_mask_decoded, annotation['bbox'], image_size )
                    ann_bbox = [int(bbox[0]),int(bbox[1]),int(bbox[2]),int(bbox[3])]
                    ann_segment = encoded_mask
                    ann_image_id = annotation['image_id']
                    ann_id = -1
                    ann_cat_id = 23+i
                    if len(contours)>4:
                        new_data = {'segmentation': [contours], # encoded_mask,
                                    'area': ann_area,
                                    'image_id': ann_image_id,
                                    'id': ann_id,
                                    'category_id': ann_cat_id,
                                    'bbox': ann_bbox,
                                    'iscrowd': 0}
                        new_person_ann.append(new_data)

100%|██████████| 25053/25053 [05:14<00:00, 79.59it/s]


In [28]:
x = new_person_ann[1]


In [29]:
type(x['segmentation'])

list

# till here I have all human data and parts segmented and annotated
# next step is to include the other classes

In [30]:
new_data_ann = []
new_data_image_list = []
new_data_images = []
for annotation in tqdm(data_df['annotations'][0]):
    if annotation['category_id'] in custom_category_coco_category and annotation['category_id']>0:
        ann_segment = annotation['segmentation']
        ann_bbox = [int(annotation['bbox'][0]),int(annotation['bbox'][1]),int(annotation['bbox'][2]),int(annotation['bbox'][3])]
        ann_area = annotation['area']
        ann_image_id = annotation['image_id']
        ann_id = annotation['id']
        ann_iscrowd = annotation['iscrowd']
        ann_cat_id = custom_category_coco_category.index(annotation['category_id'])
        new_data = {'segmentation': ann_segment,
                    'area': ann_area,
                    'image_id': ann_image_id,
                    'id': ann_id,
                    'category_id': ann_cat_id,
                    'bbox': ann_bbox,
                    'iscrowd': ann_iscrowd}
        new_person_ann.append(new_data)
        new_data_image_list.append(ann_image_id)

100%|██████████| 291875/291875 [00:02<00:00, 144417.80it/s]


In [31]:
new_data_images = [image for image in data_df['images'][0] if image['id'] in new_data_image_list]

# Updating indices

In [32]:
final_images = new_person_images + new_data_images
final_annotations = new_person_ann + new_data_ann

In [33]:
nil = []
for image in tqdm(final_images):
    if image["id"] not in all_images_list:
        nil.append[image["id"]]
len(nil)

100%|██████████| 34882/34882 [00:09<00:00, 3520.57it/s]


0

In [34]:
final_annotations[0].keys()

dict_keys(['segmentation', 'area', 'image_id', 'id', 'category_id', 'bbox', 'iscrowd'])

In [35]:
for i in tqdm(range(len(final_annotations))):
    final_annotations[i]['id'] = i+1


100%|██████████| 250523/250523 [00:00<00:00, 2012509.83it/s]


In [36]:
nil = []
for image in tqdm(final_images):
    if image["id"] not in all_images_list:
        nil.append[image["id"]]
len(nil)

100%|██████████| 34882/34882 [00:09<00:00, 3520.47it/s]


0

# Saving humans parts data

In [37]:

new_human_part_data = {
    'images': new_person_images, 
    'annotations': new_person_ann, 
    'categories': custom_categories
}

In [38]:
class MyEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__
new_human_part_data_json = MyEncoder().encode(new_human_part_data)

In [39]:
with open('/home/ubuntu/dormakaba/coco/annotations/custom_ann_human_parts_valid2014_v10.json', 'w') as f:
    json.dump(new_human_part_data_json, f)

# Saving non humans data


In [40]:

new_non_human_data = {
    'images': new_data_images, 
    'annotations': new_data_ann, 
    'categories': custom_categories
}

In [41]:
class MyEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__
new_non_human_data_json = MyEncoder().encode(new_non_human_data)

In [42]:
with open('/home/ubuntu/dormakaba/coco/annotations/custom_ann_non_human_valid2014_v10.json', 'w') as f:
    json.dump(new_non_human_data_json, f)

# saving all data

In [43]:
# IN THE version_02
new_person_data = {
    'images': final_images, 
    'annotations': final_annotations, 
    'categories': custom_categories
}

In [44]:
from json import JSONEncoder
class MyEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__
new_person_data_json = MyEncoder().encode(new_person_data)

In [45]:
with open('/home/ubuntu/dormakaba/coco/annotations/custom_ann_valid2014_v10.json', 'w') as f:
    json.dump(new_person_data_json, f)